In [60]:
import pandas as pd
import json
import os

from src.data_manipulation import data_processing

In [61]:
with open("data/liquidation_events.json", "r") as file:
    agreementLiquidatedByEvents = json.loads(file.read())

with open("data/account_token_snapshots.json", "r") as file:
    accountTokenSnapshots = json.loads(file.read())

for snapshot in accountTokenSnapshots:

    snapshot["balanceUntilUpdatedAt"] = snapshot["accountTokenSnapshot"]["balanceUntilUpdatedAt"]
    snapshot["updatedAtTimestamp"] = snapshot["accountTokenSnapshot"]["updatedAtTimestamp"]
    snapshot.pop("accountTokenSnapshot")


In [62]:
liquidation_events = agreementLiquidatedByEvents
account_token_snapshots = accountTokenSnapshots 

In [63]:
# DATA IMPORT
agreementLiquidatedByEvents = pd.DataFrame(liquidation_events)
accountTokenSnapshots = pd.DataFrame(account_token_snapshots)   

In [64]:
# TYPE FORMATTING
accountTokenSnapshots["balance"] = accountTokenSnapshots["balance_form"]
accountTokenSnapshots["totalNetFlowRate"] = accountTokenSnapshots["totalNetFlowRate_form"]
accountTokenSnapshots = accountTokenSnapshots.drop(["balance_form", "totalNetFlowRate_form"], axis=1)
agreementLiquidatedByEvents["rewardAmount"] = agreementLiquidatedByEvents["rewardAmount_form"]
agreementLiquidatedByEvents = agreementLiquidatedByEvents.drop("rewardAmount_form", axis=1)

In [65]:
# DATA WRANGLING
#agreementLiquidatedByEvents["targetAccount-token"] = agreementLiquidatedByEvents['targetAccount'] + " - " + agreementLiquidatedByEvents['token']
#agreementLiquidatedByEvents = agreementLiquidatedByEvents[[not elem for elem in agreementLiquidatedByEvents.duplicated(subset="targetAccount-token")]].reset_index()
#accountTokenSnapshots = accountTokenSnapshots[[not elem for elem in accountTokenSnapshots.duplicated(subset="id")]].reset_index()
## create index columns to merge datasets
accountTokenSnapshots[['prefix', 'targetAccount', 'token', 'transactionHash', 'logIndex']] = accountTokenSnapshots['id'].str.split('-', -1, expand=True)



In [66]:
# filter out ATS duplicates 
accountTokenSnapshots = accountTokenSnapshots.drop_duplicates(["blockNumber", "targetAccount", "token"])

In [67]:
# merge datasets
merged_data = agreementLiquidatedByEvents.merge(accountTokenSnapshots, how="left", on=["blockNumber", "targetAccount", "token"])

In [ ]:

# select only relevant columns 
merged_data = merged_data[["liquidatorAccount", "rewardAmount","timestamp", "balanceUntilUpdatedAt", "updatedAtTimestamp", "totalNetFlowRate"]]

# rename columns 
merged_data.rename(columns = {'timestamp':'liquidationTimestampUNIX', 'updatedAtTimestamp': 'dateCreatedTimestampUNIX', 'balanceUntilUpdatedAt': 'startingBalance'}, inplace = True)

# calculations
## convert datatime column into unix format
merged_data["liquidationTimestampUNIX"] = merged_data["liquidationTimestampUNIX"].astype(int).astype(str).str[:10].astype(int)

merged_data["balanceSendAway"] = (merged_data["liquidationTimestampUNIX"] - merged_data["dateCreatedTimestampUNIX"].astype(int)) * merged_data["totalNetFlowRate"]
merged_data["balanceAtLiquidation"] = merged_data["balanceSendAway"] + merged_data["startingBalance"]
merged_data["responseTime_sec"] = merged_data["balanceAtLiquidation"] / merged_data["totalNetFlowRate"]

# feature engineering
merged_data["liquidationTimestampDatetime"] = pd.to_datetime(merged_data['liquidationTimestampUNIX'], unit='s')
merged_data["date"] = merged_data["liquidationTimestampDatetime"].dt.date
merged_data["hourOfDay"] = merged_data["liquidationTimestampDatetime"].dt.hour
merged_data["dateHour"] = merged_data["liquidationTimestampDatetime"].dt.year.astype(str) + "-" + \
    merged_data["liquidationTimestampDatetime"].dt.month.astype(str).str.zfill(2) + "-" + \
    merged_data["liquidationTimestampDatetime"].dt.day.astype(str).str.zfill(2) + " " + \
    merged_data["liquidationTimestampDatetime"].dt.hour.astype(str).str.zfill(2) + ":00"

In [3]:
merged_data = data_processing(
    liquidation_events=agreementLiquidatedByEvents, 
    account_token_snapshots=accountTokenSnapshots
    )